In [1]:
from onnxruntime.quantization import QuantFormat, QuantType, quantize_static
import resnet50_data_reader

In [2]:
dr = resnet50_data_reader.ResNet50DataReader(
        "test_images", "efficientvit_b0.r224_in1k_simplified.onnx"
    )

In [3]:
quantize_static(
        "efficientvit_b0.r224_in1k_simplified.onnx",
        "efficientvit_b0.r224_in1k_simplified_quantized_static.onnx",
        dr,
        quant_format=QuantFormat.QDQ,
        per_channel=True,
        weight_type=QuantType.QInt8,
    )

In [4]:
import numpy as np
import onnxruntime as ort
from PIL import Image
from urllib.request import urlopen

#define the priority order for the execution providers

# prefer CUDA Execution Provider over CPU Execution Provider
EP_list = ['CPUExecutionProvider']

# Load ONNX model
session = ort.InferenceSession("efficientvit_b0.r224_in1k_simplified_quantized_static.onnx", providers=EP_list)

# session.set_providers(['CPUExecutionProvider'])

# Load an image
img = Image.open(urlopen('https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'))
img = img.convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img).astype(np.float32)

# Convert data to the shape the ONNX model expects
input_data = np.transpose(img_np, (2, 0, 1))  # Convert to (C, H, W)
input_data = np.expand_dims(input_data, axis=0)  # Add a batch dimension

input_data.shape

# Get input name from the model
input_name = session.get_inputs()[0].name



In [5]:
%%timeit
# Perform inference
output = session.run(None, {input_name: input_data})

10 ms ± 71.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
